In [1]:
# Создание датасета
import torch
from listen_ipa_dataset import *
import json
with open('./data/wikitionary_data/raw_data_stage_07.1_non_slash.json', 'r', encoding="utf8") as f:
    all_lines = f.readlines()
simple_pairs = [json.loads(line) for line in all_lines]
stage_07_dataset = ListenIPADataset(simple_pairs, amplify=1, bias=0)
torch.save(stage_07_dataset, './data/dataset/listen_ipa_dataset_0.7.1_non_slash.pt')
del all_lines, simple_pairs, stage_07_dataset

In [3]:
# Установки библиотек при необходимости
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install librosa==0.10.2.post1
!pip install soundfile==0.12.1
!pip install numpy==1.23.5
!pip install matplotlib==3.9.0
!pip install tqdm

Looking in indexes: https://download.pytorch.org/whl/cu121
     ---------------------------------------- 0.0/2.4 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.4 GB 2.9 MB/s eta 0:14:02
     ---------------------------------------- 0.0/2.4 GB 6.3 MB/s eta 0:06:25
     ---------------------------------------- 0.0/2.4 GB 12.9 MB/s eta 0:03:09
     ---------------------------------------- 0.0/2.4 GB 14.3 MB/s eta 0:02:50
     ---------------------------------------- 0.0/2.4 GB 13.6 MB/s eta 0:02:58
     ---------------------------------------- 0.0/2.4 GB 18.9 MB/s eta 0:02:08
     ---------------------------------------- 0.0/2.4 GB 18.9 MB/s eta 0:02:08
     ---------------------------------------- 0.0/2.4 GB 27.6 MB/s eta 0:01:28
     ---------------------------------------- 0.0/2.4 GB 40.9 MB/s eta 0:00:59
     ---------------------------------------- 0.0/2.4 GB 81.8 MB/s eta 0:00:30
     ---------------------------------------- 0.0/2.4 GB 81.8 MB/s eta 0:00:30
   

ERROR: Invalid requirement: '#==0.8.1'


  Using cached soundfile-0.12.1-py2.py3-none-win_amd64.whl.metadata (14 kB)
  Using cached cffi-1.16.0-cp311-cp311-win_amd64.whl.metadata (1.5 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
Using cached soundfile-0.12.1-py2.py3-none-win_amd64.whl (1.0 MB)
Using cached cffi-1.16.0-cp311-cp311-win_amd64.whl (181 kB)
Using cached pycparser-2.22-py3-none-any.whl (117 kB)
  Using cached numpy-1.23.5-cp311-cp311-win_amd64.whl.metadata (2.3 kB)
Using cached numpy-1.23.5-cp311-cp311-win_amd64.whl (14.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.3
    Uninstalling numpy-1.26.3:
      Successfully uninstalled numpy-1.26.3
  Using cached matplotlib-3.9.0-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached contourpy-1.2.1-cp311-cp311-win_amd64.whl.metadata (5.8 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.53.0-cp311-cp311-win_amd64.whl.metadata (165 kB)
  Using cached kiwisolver-1.4.

In [3]:
# Импорт torch, переключение его на cuda, импорт датасета и разденение его на выборки
import torch
from torch.utils.data import random_split
from tqdm import tqdm
from listen_ipa_dataset import *

from random import randint
from criteria_to_symbols import *
from time import time


torch.set_printoptions(precision=8)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
used_dataset = torch.load('./data/dataset/listen_ipa_dataset_0.7.1_non_slash.pt')
train_size = int(0.6 * len(used_dataset))
test_size = len(used_dataset) - train_size
torch.set_default_device("cpu")
train_dataset, test_dataset = random_split(used_dataset, [train_size, test_size], generator=torch.Generator().manual_seed(3))
torch.set_default_device(device)
print(len(train_dataset), len(test_dataset))

cuda:0
45943 30629


In [4]:
# Импорт класса модели
from model import *

In [5]:
# Переметры нейросети
kernel_size = 15 # 35
num_layers = 4 # 4
num_epochs = 100
audio_input_size = len(used_dataset[0]["audio"][0])
old_version = False

In [6]:
#Отобразить модель (лучше не делать - уже готова)
# import torchviz

model_test = ListenIPA(audio_input_size, CRITERIA_LEN, kernel_size, num_layers, old_version)

hidden = model_test.init_hidden()
output, _ = model_test(torch.tensor([[[0.0 for i in range(audio_input_size)]]], dtype=torch.complex64), hidden)
# dot = torchviz.make_dot(output, params=dict(model_test.named_parameters())).render("model_structure2", format="png")
del model_test

In [7]:
# Модель
torch.set_default_device(device)
model = ListenIPA(audio_input_size, CRITERIA_LEN, kernel_size, num_layers, old_version)
loading_epoch = 0
continue_n = -1
n = 0
epoch_times = {}
epoch_loss = {}
just_stopped = False
model

ListenIPA(
  (conv1): Conv1d(1, 1, kernel_size=(15,), stride=(2,), padding=(7,))
  (conv2): Conv1d(1, 1, kernel_size=(15,), stride=(2,), padding=(7,), bias=False)
  (initial_linears2): Sequential(
    (0): Linear(in_features=258, out_features=129, bias=False)
    (1): Linear(in_features=129, out_features=129, bias=False)
  )
  (recc): GRU(129, 129, num_layers=4)
  (final_linears1): Sequential(
    (0): Linear(in_features=129, out_features=22, bias=False)
  )
)

In [78]:
# Оптимизатор и фукнкция потерь
import torch.optim as optim
torch.set_default_device(device)

criterion = nn.MSELoss()
learning_rate = 0.00125
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [242]:
# Тренировка
just_stopped = False
last_outputs_priority = 0.0

pattern_outputs_priority = 100
thinking_hyperbola = 0.08 #0.0125

base_value = 0
answer_value = 0.0

sound_max = used_dataset.criteria_max()
sound_min = used_dataset.criteria_min()
sound_mid = (sound_max + sound_min) / 2
sound_to_answer = 0.45

# close_answer_punish_mult = 1
thinking_mult = 0.25

def crit_deg(v): return max((0.0012/(v*v+0.02))-0.01, 0.0)

def adapt_to_output(label, output, prev_output=None):
    if label[CRITERIA_TAGS_ID["thi"]] == sound_max:
        cut_num = 1
        curr_crit = CRITERIA_TAGS_ID["thi"]
        label[curr_crit] = max(output[curr_crit], sound_max)
        
        if prev_output is None:
            for crit in range(cut_num, CRITERIA_LEN):
                label[crit] = output[crit]
        else:
            for crit in range(cut_num, CRITERIA_LEN):
                op = output[crit]
                prev_op = prev_output[crit]
                curr_deg = crit_deg(op-prev_op)
                label[crit] = op
                if curr_deg != 0:
                    label[crit] += curr_deg if prev_op < op else -curr_deg
    elif label[CRITERIA_TAGS_ID["spa"]] == sound_max:
        curr_crit = CRITERIA_TAGS_ID["spa"]
        cut_num = 3
        label[curr_crit] = max(output[curr_crit], sound_max)
        
        for crit in range(curr_crit):
            bl = label[crit]
            op = output[crit]
            if (op > sound_max and bl == sound_max) or (op < sound_min and bl == sound_min):
                label[crit] = op

        if prev_output is None:
            for crit in range(cut_num, CRITERIA_LEN):
                label[crit] = output[crit]
        else:
            for crit in range(cut_num, CRITERIA_LEN):
                op = output[crit]
                prev_op = prev_output[crit]
                curr_deg = crit_deg(op-prev_op)
                label[crit] = op
                if curr_deg != 0:
                    label[crit] += curr_deg if prev_op < op else -curr_deg
    else:
        cut_num = -1
        for crit in range(CRITERIA_LEN):
            bl = label[crit]
            op = output[crit]
            if (op > sound_max and bl == sound_max) or (op < sound_min and bl == sound_min):
                label[crit] = op
    return label, cut_num

def smoothing_array(v:int, v_max:int):
    if v_max == 0: return 1.0
    return np.sin(((((v + 1) * 2 / (v_max + 2)) - 1) / 2 + 0.5) * np.pi)

def smoothing_array_whole(fi):
    arr = []
    for i in range(fi):
        arr.append(smoothing_array(i, fi-1))
    return arr

# total_epoch_losses = {}
n = 0
thi = CRITERIA_TAGS_ID["thi"]
model.to(device)
model.train()
for epoch in range(loading_epoch, num_epochs):
    if epoch not in epoch_times:
        epoch_times[epoch] = 0.0
    if epoch not in epoch_loss:
        epoch_loss[epoch] = []

    for sample in tqdm(train_dataset, desc=f"Epoch #{epoch+1}"):
        if continue_n < n:
            start_time = time()
            audio = [i for i in sample['audio']]
            labels = [[j for j in i] for i in sample['labels']]
            hidden = model.init_hidden()
            outputs = []

            optimizer.zero_grad()
            for audio_frame in audio:
                if old_version:
                    output, hidden = model(torch.tensor(audio_frame, dtype=torch.complex64).unsqueeze(0).unsqueeze(0),
                                           (hidden[0].detach(), hidden[1].detach()))
                else:
                    output, hidden = model(torch.tensor(audio_frame, dtype=torch.complex64).unsqueeze(0).unsqueeze(0),
                                           hidden.detach())
                outputs.append(output)
            outputs_len = len(outputs)

            # Prioritizing last outputs (can't be a sound after the first output)
            custom_balanced = False
            custom_min = False
            n_epoch = n + epoch
            
            output_list_initial = [[n, float(i[0][0][0])] for n, i in enumerate(outputs)]
            custom_min = True
            think_output_values = [i[1] for i in output_list_initial]
            output_list_initial_min = min(think_output_values)
            output_list_initial_diff = max(think_output_values) - output_list_initial_min
            output_list_initial_diff = 0.0001 if output_list_initial_diff == 0 else output_list_initial_diff
            output_list_initial = [[n,
                ((i-output_list_initial_min)/output_list_initial_diff)*base_value - ((n/outputs_len)*last_outputs_priority)]
                    for n, i in output_list_initial]

            labels_lens = set()
            for l in labels:
                labels_lens.add(len(l))
            labels_lens = sorted(list(labels_lens))

            output_list_initials = {}
            for sound in labels_lens:
                period = outputs_len // sound
                period_start = period
                period_end = period + (period // 4)
                arr = "1"*(period_start)
                for i in range(sound):
                    arr += "1"*period
                    arr += "0"
                arr += "1"*(period_end)
                arr2 = arr[::-1].split("0")
                curr_array_id = 0
                while len(arr) != outputs_len:
                    arr2[curr_array_id] = arr2[curr_array_id][1:]
                    curr_array_id += 1
                    if curr_array_id == len(arr2):
                        curr_array_id = 0
                    arr = "0".join(arr2)
                arr2 = arr[::-1].split("0")

                a_len = len(arr2[0])
                smooth_arr = [i*2 for i in smoothing_array_whole(a_len*2)[a_len:]]+[0]
                for a in arr2[1:-1]:
                    smooth_arr += smoothing_array_whole(len(a))+[0]
                a_len = len(arr2[-1])
                smooth_arr += [i*2 for i in smoothing_array_whole(a_len*2)[:a_len]]
                
                output_list_initials[sound] = [[n, i*pattern_outputs_priority]
                                               for n, i in enumerate(smooth_arr)]

            for sound in labels_lens:
                for o in range(outputs_len):
                    output_list_initials[sound][o][1] += output_list_initial[o][1]

            labels_losses = []
            labels_smooth_losses = []
            labels_losses_simple = []
            labels_losses_avg_sounds_placement = []
            labels_losses_balances = []

            for label in labels:
                label_len = len(label)
                empt = used_dataset.thinking_empty()

                curr_pattern = sorted(output_list_initials[label_len][1:].copy(), key=lambda x: x[1])
                curr_outputs_labels = sorted([i[0] for i in curr_pattern[:label_len]])

                curr_outputs_labels_borders = [0]+curr_outputs_labels+[outputs_len]
                skips = [curr_outputs_labels_borders[i+1] - curr_outputs_labels_borders[i]
                         for i in range(label_len+1)]
                first_skip = [skips[0] // 2, skips[0] // 2 + skips[0] % 2]
                last_skip = [skips[-1] // 2 + skips[-1] % 2, skips[-1] // 2]
                skips = first_skip + skips[1:-1] + last_skip

                second_last_label_sound = list(label[-1])
                second_last_label_sound[0] = 1

                border_label = [empt, empt] + label + [second_last_label_sound, empt]
                
                smoothing = []
                for bl in range(len(border_label)-1):
                    for skip_value in range(skips[bl]):
                        prev_value = border_label[bl]
                        next_value = border_label[bl+1]
                        gradient_step = (skip_value+1)/skips[bl]
                        if next_value[0] == 1:
                            think_value = 1.0
                        else:
                            think_value = 1-(thinking_hyperbola/(1-gradient_step+thinking_hyperbola))
                        if think_value == 0:
                            think_value = answer_value
                        smooth_value = (gradient_step * 1.5)/(gradient_step + 0.5)
                        smoothing.append([think_value]+[prev_value[i]*(1-smooth_value) + next_value[i]*smooth_value
                                        for i in range(1, CRITERIA_LEN)])
                labels_smooth_losses.append(smoothing)

                curr_whole_loss = 0.0
                label_sound_id = 0
                smooth_adapted_target_output = []
                for curr_n in range(outputs_len):
                    if curr_n in curr_outputs_labels:
                        for characteristic in range(CRITERIA_LEN):
                            curr_bop = float(outputs[curr_n][0][0][characteristic])
                            curr_bop = sound_max if curr_bop > sound_max else sound_min if curr_bop < sound_min else curr_bop
                            curr_bop = label[label_sound_id][characteristic] - curr_bop
                            curr_whole_loss += abs(curr_bop)
                        label_sound_id += 1
                    else:
                        curr_whole_loss += abs(sound_max - float(outputs[curr_n][0][0][0]))

                labels_losses_simple.append(curr_whole_loss)
                labels_losses_balances.append(curr_outputs_labels)
                labels_losses_avg_sounds_placement.append((sum(curr_outputs_labels) / label_len) / outputs_len)
            
            if len(labels) == 1:
                min_labels_losses_id = 0
            else:
                min_labels_losses_id = min(range(len(labels_losses_simple)), key=labels_losses_simple.__getitem__)
            adapted_label = labels_smooth_losses[min_labels_losses_id] # Smoothed version
            curr_avg_sound_place = labels_losses_avg_sounds_placement[min_labels_losses_id]
            curr_balance = labels_losses_balances[min_labels_losses_id]
            
            # TRAINING
            # hidden = model.init_hidden()
            curtom_prev = False
            frames_loss = 0
            used_frames = 0
            optimizer.zero_grad()
            hidden = model.init_hidden()
            for frame, audio_frame in enumerate(audio):
                if old_version:
                    output, hidden = model(torch.tensor(audio_frame, dtype=torch.complex64).unsqueeze(0).unsqueeze(0),
                                           (hidden[0].detach(), hidden[1].detach()))
                else:
                    output, hidden = model(torch.tensor(audio_frame, dtype=torch.complex64).unsqueeze(0).unsqueeze(0),
                                           hidden.detach())
                loss = criterion(output, torch.tensor([[adapted_label[frame]]]))

                must_to_answer = adapted_label[frame][thi] == answer_value
                must_to_answer_custom = frame in [0]  # outputs_len-1
                is_answering_close = float(output[0][0][thi]) < sound_to_answer
                # used_frames = outputs_len
                if not(must_to_answer or must_to_answer_custom or is_answering_close):
                    loss *= thinking_mult
                    used_frames += thinking_mult
                else:
                    used_frames += 1
                
                frames_loss += loss.item()
                loss.backward(retain_graph=True)
            optimizer.step()

            
            sample_time = time() - start_time
            epoch_times[epoch] += sample_time
            epoch_loss[epoch] += [[sample_time, frames_loss/used_frames, curr_avg_sound_place, curr_balance, outputs_len]]

        n += 1
    # total_epoch_losses[epoch] = epoch_loss[epoch]

    # Print epoch loss
    # print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/n:.4f}, Samples: {n}')
    
    # Save the model checkpoint
    model_path = f'./model_epoch_{epoch+1}.pth'
    torch.save(model.state_dict(), model_path)
    print(f'Model saved to {model_path}')
    loading_epoch += 1
    continue_n = -1
    n = 0
    # break

Epoch #2: 100%|██████████| 45943/45943 [4:23:45<00:00,  2.90it/s]  


Model saved to ./model_epoch_2.pth


Epoch #3:   1%|          | 503/45943 [05:48<8:44:31,  1.44it/s] 


KeyboardInterrupt: 

In [243]:
# Время обучения эпох в часах
for i in epoch_loss:
    print(i+1, epoch_times[i]/60/60)

1 8.727760089039803
2 8.72372738275263
3 0.09607303573025597


In [244]:
# Продолжение обучения
continue_n = n
just_stopped = True
print(f"Last epoch: {loading_epoch+1}, Continue N: {continue_n}")

Last epoch: 3, Continue N: 503


In [245]:
# Время обучения и какие последние шаблоны правильных ответов были использованы
curr_epoch_max = max(epoch_loss)
print(f'Epoch #{curr_epoch_max+1}')
for i in epoch_loss[curr_epoch_max][-5:]: #28
    # print(round(i[0], 3), end=" ")
    print(round(i[1], 3), end=" ")
    print(round(i[2], 3), end=" ")
    print(i[3], end=" ")
    print(i[4])

Epoch #3
0.061 0.583 [8, 12, 16, 20, 23, 26] 30
0.061 0.56 [15, 22, 29, 36, 43, 50, 56] 64
0.058 0.558 [14, 21, 28, 35, 42, 48, 54] 62
0.077 0.535 [11, 16, 21, 26, 31, 36, 41, 46, 51, 56, 61, 66] 72
0.065 0.539 [15, 23, 30, 37, 44, 51, 58, 65, 72, 79] 88


In [325]:
# Демонстрация
think_test = 0.5  # default 0.5

# torch.set_default_device(device)
model.eval()
all_outputs = []
all_labels = []
sound_max = used_dataset.criteria_max()
sound_min = used_dataset.criteria_min()
sound_mid = (sound_max + sound_min) / 2

# with torch.no_grad():  # Disable gradient calculations
test_n = 0
if just_stopped:
    rand_test_n = 0
    just_stopped = False
else:
    rand_test_n = randint(0, 1000)

    rand_test_n = 0 # For showcasing
# rand_test_n = 258
print(rand_test_n, end=" ")
# for sample in test_dataset:
for sample in test_dataset:
    # if test_n == 60:
    if test_n == rand_test_n or True:
        audio = [torch.tensor(i, dtype=torch.complex64) for i in sample['audio']]
        labels = [[torch.tensor(j, dtype=torch.float32) for j in i] for i in sample['labels']]
        print(sample['audio_id'], len(audio))

        hidden = model.init_hidden()

        outputs = []
        # hiddens = []
        model.zero_grad()
        for audio_frame in audio:
            audio_frame = audio_frame.unsqueeze(0).unsqueeze(0)
            output, hidden = model(audio_frame, hidden)
            outputs.append(output)
            # hiddens.append((hidden[0].to(device), hidden[1].to(device)))
        # all_outputs.append(torch.stack(outputs))
        # all_labels.append(labels)
        
        collected_outputs = [[float(j) for j in i[0][0]] for i in outputs]
        print("Output_filtered")
        curr_s = output_to_symbols(collected_outputs, think_test)
        outputs_thought = [float(i[0][0][0]) for i in outputs]
        outputs_thought = len([i for i in outputs_thought if i <= think_test])
        print(f"\t{outputs_thought}\t{curr_s}")
        print("ID", "     ".join([CRITERIA_NAMES[i][1] for i in range(CRITERIA_LEN)]), sep="\t")
        for enum, i in enumerate(collected_outputs):
            if float(i[0]) <= think_test:
                print(enum, "\t".join([f"{j:.3f}".replace(".", ",") for j in i]), sep="\t")
        print()
        print("Label")
        collected_label = [[j for j in i] for i in sample['labels'][0]]
        # print(collected_label)
        print(f"\t\t{sample['labels_text'][0][1][1:-1]}")
        curr_s = output_to_symbols(collected_label, think_test)
        print(f"\t{len(sample['labels'][0])}\t{curr_s}")
        print("ID", "     ".join([CRITERIA_NAMES[i][1] for i in range(CRITERIA_LEN)]), sep="\t")
        for enum, i in enumerate(collected_label):
            print(enum, "\t".join([f"{j:.3f}".replace(".", ",") for j in i]), sep="\t")
        print()
        print("Output")
        curr_s = output_to_symbols(collected_outputs, 100)
        outputs_thought = [float(i[0][0][0]) for i in outputs]
        outputs_thought = len([i for i in outputs_thought if i <= 100])
        print(f"\t{outputs_thought}\t{curr_s}")
        print("ID", "     ".join([CRITERIA_NAMES[i][1] for i in range(CRITERIA_LEN)]), sep="\t")
        for enum, i in enumerate(collected_outputs):
            print(enum, "\t".join([f"{j:.3f}".replace(".", ",") for j in i]), sep="\t")
        print()
        test_n +=1
        # break
        if test_n > 10:
            break
    else:
        test_n +=1
        
# all_outputs = torch.cat(all_outputs, dim=0)
# all_labels = torch.cat(all_labels, dim=0)

0 18261 33
Output_filtered
	14	ɪ̈ɨɨɨɨɨɨɨɨɨɨɪ̈ɨɨ
ID	thi     spa     lon     str     con     nas     plo     sib     tri     lat     lbz     bil     den     alv     ret     pal     vel     uvu     pha     glo     asp     vls
13	0,493	-0,007	0,018	0,147	0,682	0,078	0,219	0,052	0,071	0,053	0,153	0,183	0,063	0,329	0,029	0,303	0,276	0,020	-0,000	0,031	0,009	0,144
14	0,484	-0,007	0,019	0,154	0,688	0,075	0,220	0,051	0,073	0,057	0,158	0,184	0,063	0,330	0,030	0,308	0,281	0,022	-0,000	0,031	0,010	0,142
15	0,478	-0,008	0,023	0,162	0,690	0,081	0,208	0,044	0,073	0,063	0,165	0,175	0,060	0,330	0,029	0,323	0,287	0,023	-0,000	0,032	0,010	0,116
16	0,475	-0,008	0,028	0,165	0,697	0,089	0,197	0,044	0,071	0,066	0,166	0,159	0,058	0,338	0,031	0,342	0,288	0,024	-0,000	0,031	0,011	0,113
17	0,472	-0,007	0,031	0,170	0,701	0,093	0,191	0,042	0,072	0,068	0,168	0,149	0,056	0,343	0,030	0,353	0,291	0,024	-0,000	0,031	0,011	0,107
18	0,472	-0,007	0,034	0,172	0,704	0,097	0,186	0,039	0,073	0,069	0,170	0,141	0,054	0,346	0,03

In [324]:
# Проверка точности

def calc_diff(list1, list2, length):
    curr_diff = []
    for crit in range(length):
        curr_diff.append(abs(list2[crit] - list1[crit]))
    return curr_diff

accuracies = []
accuracies_answer_all = []
accuracies_answer_crit = []
accuracies_think = []
model.eval()
for sample in tqdm(test_dataset, desc=f"Accuracy test"):
    audio = [torch.tensor(i, dtype=torch.complex64) for i in sample['audio']]
    labels = sample['labels']

    hidden = model.init_hidden()
    outputs = []
    model.zero_grad()
    for audio_frame in audio:
        audio_frame = audio_frame.unsqueeze(0).unsqueeze(0)
        output, hidden = model(audio_frame, hidden)
        outputs.append(output)
    outputs_len = len(audio)
    
    collected_outputs = [[float(j) for j in i[0][0]] for i in outputs]
    collected_outputs_sort_think = [i[0] for i in sorted(enumerate(collected_outputs), key=lambda x: x[1][0])]

    labels_accuracies = []
    labels_accuracies_answer_all = []
    labels_accuracies_answer_crit = []
    labels_accuracies_think = []
    for curr_n, label in enumerate(labels):
        curr_output_filter = sorted(collected_outputs_sort_think[:len(label)])
        curr_label_accuracy = 0.0
        curr_label_accuracy_answer = [[0.0, 0] for _ in range(CRITERIA_LEN)]
        curr_label_accuracy_think = 0.0
        l = 0
        for o in range(outputs_len):
            if o in curr_output_filter:
                if label[l][1] == 1:
                    diffs = calc_diff(collected_outputs[o][:3], label[l][:3], 3)
                    for d in range(3):
                        curr_label_accuracy_answer[d][0] += diffs[d]
                        curr_label_accuracy_answer[d][1] += 1
                else:
                    diffs = calc_diff(collected_outputs[o], label[l], CRITERIA_LEN)
                    for d in range(CRITERIA_LEN):
                        curr_label_accuracy_answer[d][0] += diffs[d]
                        curr_label_accuracy_answer[d][1] += 1
                l += 1
            else:
                curr_label_accuracy_think += calc_diff(collected_outputs[o][:1], [1], 1)[0]
        curr_label_accuracy_answer_accuracy = sum([i[0] for i in curr_label_accuracy_answer])
        curr_label_accuracy_answer_values = sum([i[1] for i in curr_label_accuracy_answer])
        curr_label_accuracy_answer_all = curr_label_accuracy_answer_accuracy / curr_label_accuracy_answer_values
        labels_accuracies_answer_all.append([curr_n, curr_label_accuracy_answer_all])
        labels_accuracies_answer_crit.append([curr_n, curr_label_accuracy_answer])
        labels_accuracies_think.append([curr_n, curr_label_accuracy_think])
        labels_accuracies.append([curr_n, curr_label_accuracy_answer_all + curr_label_accuracy_think])
    nearest_label = sorted(labels_accuracies, key=lambda x: x[1])[0]
    nearest_label_len = len(labels[nearest_label[0]])

    accuracies_answer_all.append(1-(labels_accuracies_answer_all[nearest_label[0]][1]/nearest_label_len))

    curr_accuracy_answer_crit = [1-(i/c) for i, c in labels_accuracies_answer_crit[nearest_label[0]][1] if c != 0]
    accuracies_answer_crit.append(curr_accuracy_answer_crit)

    accuracies_think.append(1-(labels_accuracies_think[nearest_label[0]][1]/(outputs_len-nearest_label_len)))
    accuracies.append(1-(nearest_label[1]/outputs_len))

accuracies_len = len(accuracies)
accuracies_sum = sum(accuracies)
print(f"Average accuracy on {accuracies_len} test samples: {accuracies_sum/accuracies_len}")
accuracies_think_len = len(accuracies_think)
accuracies_think_sum = sum(accuracies_think)
print(f"Average accuracy for think: {accuracies_think_sum/accuracies_think_len}")
accuracies_answer_len = len(accuracies_answer_all)
accuracies_answer_sum = sum(accuracies_answer_all)
print(f"Average accuracy (all criteria) for answer: {accuracies_answer_sum/accuracies_answer_len}")
crit_list = [[accuracies_answer_crit[j][i] for j in range(len(accuracies_answer_crit))] for i in range(CRITERIA_LEN)]
crit_list = [sum(i)/len(i) for i in crit_list]
print(f"Average accuracy (separate criteria) for answer:")
for i in range(CRITERIA_LEN):
    print(f'\t{[CRITERIA_NAMES[i][1], round(crit_list[i], 5)]}')

Accuracy test:   0%|          | 16/30629 [00:03<1:43:57,  4.91it/s]


KeyboardInterrupt: 

In [326]:
# Точность, точность обдумывания,
# точность отвтетов для последних элементов проверки
for i in range(-1, -30, -1):
    print(f"{round(accuracies[i], 4)}\t{round(accuracies_think[i], 4)}\t{round(accuracies_answer_all[i], 4)}")

0.6552	0.6161	0.9799
0.6667	0.6141	0.9646
0.6645	0.6291	0.9786
0.5835	0.5325	0.9663
0.6153	0.5529	0.9717
0.6521	0.6286	0.9191
0.6369	0.6012	0.9498
0.5916	0.5306	0.978
0.6	0.574	0.9382
0.6717	0.6273	0.9676
0.6198	0.5894	0.9607
0.6065	0.5534	0.9785
0.673	0.619	0.9767
0.6116	0.5606	0.9584
0.6789	0.6339	0.9745
0.6948	0.648	0.9571


IndexError: list index out of range

In [327]:
# Общая точность, общая точность обдумывания,
# общая точность для критериев (всех вместе и по отдельности)
crit_list = [[accuracies_answer_crit[j][i] for j in range(len(accuracies_answer_crit))] for i in range(CRITERIA_LEN)]
crit_list = [sum(i)/len(i) for i in crit_list]

sum(accuracies)/len(accuracies), \
sum(accuracies_think)/len(accuracies_think), \
sum(accuracies_answer_all)/len(accuracies_answer_all), \
[[CRITERIA_NAMES[i][1], round(crit_list[i], 5)] for i in range(CRITERIA_LEN)]

(0.6388899568082647,
 0.5944172378841783,
 0.9637409208167076,
 [['thi', 0.53129],
  ['spa', 0.99763],
  ['lon', 0.90604],
  ['str', 0.72485],
  ['con', 0.69896],
  ['nas', 0.86144],
  ['plo', 0.64319],
  ['sib', 0.84683],
  ['tri', 0.88039],
  ['lat', 0.91379],
  ['lbz', 0.74737],
  ['bil', 0.79027],
  ['den', 0.9016],
  ['alv', 0.53491],
  ['ret', 0.9368],
  ['pal', 0.52307],
  ['vel', 0.61838],
  ['uvu', 0.95541],
  ['pha', 0.99972],
  ['glo', 0.96978],
  ['asp', 0.97779],
  ['vls', 0.63454]])

In [328]:
# Загрузка существующей модели
loading_epoch = 2
old_version = False

continue_n = -1
n = -1
epoch_times = {}
epoch_loss = {}
torch.set_default_device(device)
model_path = f'./model{"" if old_version else "_new"}_epoch_{loading_epoch}.pth'
print(f"Loading epoch #{loading_epoch} model from {model_path}")
model = ListenIPA(audio_input_size, CRITERIA_LEN, kernel_size, num_layers, old_version)
model.load_state_dict(torch.load(model_path))
model.eval()

Loading epoch #2 model from ./model_new_epoch_2.pth


ListenIPA(
  (conv1): Conv1d(1, 1, kernel_size=(15,), stride=(2,), padding=(7,))
  (conv2): Conv1d(1, 1, kernel_size=(15,), stride=(2,), padding=(7,), bias=False)
  (initial_linears2): Sequential(
    (0): Linear(in_features=258, out_features=129, bias=False)
    (1): Linear(in_features=129, out_features=129, bias=False)
  )
  (recc): GRU(129, 129, num_layers=4)
  (final_linears1): Sequential(
    (0): Linear(in_features=129, out_features=22, bias=False)
  )
)